## Introduction to the Sample Final Test

Dear Students,

Welcome to the sample final test for our laboratory course. This test is designed to assess your understanding and application of the concepts and techniques we have covered throughout the semester. 

Instructions:

Read Each Question Carefully: Ensure you understand what is being asked before you start coding.

Write Clean and Commented Code: Your code should be well-organized and include comments explaining your logic.

Test Your Code: Make sure to test your code with different inputs to ensure it works correctly.

Conclusions: Make final decisions, decide about the statistical and practical significance. 

Resources:

You are allowed to use your notes, textbooks, and online resources to help you complete the test. 

**Please be advised that the use of any Generative AI (GenAI) tools is strictly prohibited during this test. This includes, but is not limited to, tools that generate code, text, or any other form of content based on AI algorithms.**

Collaboration with classmates is not permitted. This test is an individual assessment of your skills.

I encourage you to take your time and approach each question methodically. This test is an opportunity to demonstrate your proficiency and understanding of the material. 

Best regards,

Karol
/Mathematical Statistics 2024/2025/



# Task 1: Verify the Hypothesis

Objective: Verify the hypothesis that the salaries of professors working in theoretical departments (B) are much lower than those working in applied departments (A).

In [2]:
import pandas as pd

# Load the Salaries dataset from the URL
url = "https://vincentarelbundock.github.io/Rdatasets/csv/carData/Salaries.csv"
salaries = pd.read_csv(url)

# Filter the data based on the department type
theoretical_salaries = salaries[salaries['discipline'] == 'B']['salary']
applied_salaries = salaries[salaries['discipline'] == 'A']['salary']

# Display the first few rows of the dataset
print(salaries.head())

   rownames      rank discipline  yrs.since.phd  yrs.service   sex  salary
0         1      Prof          B             19           18  Male  139750
1         2      Prof          B             20           16  Male  173200
2         3  AsstProf          B              4            3  Male   79750
3         4      Prof          B             45           39  Male  115000
4         5      Prof          B             40           41  Male  141500


In [3]:
from scipy.stats import boxcox 
import numpy as np

theoretical_salaries, _ = boxcox(theoretical_salaries)
applied_salaries, _ = boxcox(applied_salaries)

theoretical_salaries[:10], applied_salaries[:10]

(array([3.66954634, 3.67913599, 3.64179825, 3.66035959, 3.6701172 ,
        3.65194726, 3.67958464, 3.67209018, 3.6621054 , 3.66583017]),
 array([2.36084479, 2.36225032, 2.36291309, 2.35968491, 2.36077874,
        2.36007341, 2.36152599, 2.35813635, 2.3610553 , 2.36280481]))

In [4]:
from scipy.stats import shapiro

statistic, p_value = shapiro(theoretical_salaries)
print(f"For theoretical salaries: {statistic, p_value}")
statistic, p_value = shapiro(applied_salaries)
print(f"For applied salaries: {statistic, p_value}")

For theoretical salaries: (0.9891412860397887, 0.1021670726374912)
For applied salaries: (0.9850858810631471, 0.05130286095942916)


In [5]:
from scipy.stats import ttest_ind

result = ttest_ind(theoretical_salaries, applied_salaries, alternative="less")
alpha = 0.05

if result.pvalue < alpha:
    print("Reject NULL hypothesis")
else:
    print("Do not reject null hypotesis")
print(result)

Do not reject null hypotesis
TtestResult(statistic=1462.3367783497624, pvalue=1.0, df=395.0)


In [6]:
from scipy.stats import mannwhitneyu

# Mann-Whitney U test
u_statistic, p_value = mannwhitneyu(theoretical_salaries, applied_salaries, alternative='less')

print("Mann-Whitney U Test statistic:", u_statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("Reject the null hypothesis: Theoretical salaries are significantly lower.")
else:
    print("Fail to reject the null hypothesis: No significant difference.")


Mann-Whitney U Test statistic: 39096.0
p-value: 1.0
Fail to reject the null hypothesis: No significant difference.


In [7]:
import pingouin as pg

cohen_d = pg.compute_effsize(salaries[salaries['discipline'] == 'B']['salary'], salaries[salaries['discipline'] == 'A']['salary'], 
                             eftype='cohen')
print("Cohen's d:", cohen_d)

if abs(cohen_d) < 0.2:
    effect_size = "small"
elif abs(cohen_d) < 0.5:
    effect_size = "medium"
else:
    effect_size = "large"

print(f"The effect size (Cohen's d) is {cohen_d:.2f}, which is considered a {effect_size} effect.")

Cohen's d: 0.3164765707641339
The effect size (Cohen's d) is 0.32, which is considered a medium effect.


# Task 2: Verify the Hypothesis

Objective: Verify if the proportion of higher rank professors (associate and full professors) is significantly different between male and female scientists.

In [8]:
salaries.head()

,rownames,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


In [9]:
salaries["sex"].unique()

array(['Male', 'Female'], dtype=object)

In [10]:
high_rank = salaries[(salaries["rank"] == "Prof") | (salaries["rank"] == "AssocProf")]
high_rank_male = high_rank[high_rank["sex"] == "Male"]["salary"]
high_rank_female = high_rank[high_rank["sex"] == "Female"]["salary"]

high_rank_male[:10], high_rank_female[:10]

(0     139750
 1     173200
 3     115000
 4     141500
 5      97000
 6     175000
 7     147765
 8     119250
 10    119800
 14    104800
 Name: salary, dtype: int64,
 9      129000
 19     137000
 24      74830
 47     151768
 48     140096
 63     103613
 68     111512
 84     122960
 103    127512
 114    105000
 Name: salary, dtype: int64)

In [11]:
high_rank_male, _ = boxcox(high_rank_male)
high_rank_female, _ = boxcox(high_rank_female)

In [12]:
statistic, p_value = shapiro(high_rank_male)
print(f"For male salaries: {statistic, p_value}")
statistic, p_value = shapiro(high_rank_female)
print(f"For female salaries: {statistic, p_value}")

For male salaries: (0.9948767310140005, 0.41316152690894925)
For female salaries: (0.9742435305240527, 0.6975578990422144)


In [13]:
result = ttest_ind(high_rank_male, high_rank_female)
alpha = 0.05

if result.pvalue < alpha:
    print("Reject NULL hypothesis")
else:
    print("Do not reject null hypotesis")
print(result)

Reject NULL hypothesis
TtestResult(statistic=-86.59828128940755, pvalue=5.0605629281314653e-228, df=328.0)


# Task 3: Verify the Hypothesis

Objective: Verify if the salaries of professors are significantly different based on rank, gender, and discipline, and check for interactions between these groups.

In [19]:
salaries.head()

,rownames,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


In [28]:
def shapiro_test(group):
    stat, p_value = shapiro(group['salary'].dropna())
    return pd.Series({'W-statistic': stat, 'p-value': p_value})

shapiro_results = salaries.groupby(['rank', 'sex', "discipline"]).apply(shapiro_test).reset_index()

# Display the results
print(shapiro_results)

         rank     sex discipline  W-statistic   p-value
0   AssocProf  Female          A     0.862555  0.269460
1   AssocProf  Female          B     0.634562  0.001173
2   AssocProf    Male          A     0.878440  0.011282
3   AssocProf    Male          B     0.966780  0.415597
4    AsstProf  Female          A     0.869869  0.225684
5    AsstProf  Female          B     0.889298  0.353583
6    AsstProf    Male          A     0.940885  0.300046
7    AsstProf    Male          B     0.941189  0.045788
8        Prof  Female          A     0.933578  0.549195
9        Prof  Female          B     0.973714  0.922980
10       Prof    Male          A     0.952176  0.000259
11       Prof    Male          B     0.978489  0.043536


C:\Users\User\AppData\Local\Temp\ipykernel_5304\2151538707.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  shapiro_results = salaries.groupby(['rank', 'sex', "discipline"]).apply(shapiro_test).reset_index()


In [30]:
from scipy.stats import levene

# Create a combined factor for gender and Diet
salaries['rank_sex'] = salaries['rank'] + '_' + salaries['sex']

# Perform Levene's test
groups = [group['salary'].dropna().values for name, group in salaries.groupby('rank_sex')]
stat, p_value = levene(*groups)

# Display the results
print(f"Levene's test statistic: {stat}")
print(f"p-value: {p_value}")

Levene's test statistic: 17.276667237021307
p-value: 1.864247358728361e-15


In [33]:
import statsmodels.api as sm
from statsmodels.formula.api import glm

model = glm('salary ~ rank * sex * discipline', data=salaries, family=sm.families.Gaussian()).fit()

# View the summary
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 salary   No. Observations:                  397
Model:                            GLM   Df Residuals:                      385
Model Family:                Gaussian   Df Model:                           11
Link Function:               Identity   Scale:                      5.1856e+08
Method:                          IRLS   Log-Likelihood:                -4540.4
Date:                Sun, 26 Jan 2025   Deviance:                   1.9965e+11
Time:                        11:03:50   Pearson chi2:                 2.00e+11
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5486
Covariance Type:            nonrobust                                         
                                                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [24]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

salaries_copy = salaries.copy()
salaries_copy["rank"] = salaries_copy["rank"].astype("category")
salaries_copy["sex"] = salaries_copy["sex"].astype("category")
salaries_copy["discipline"] = salaries_copy["discipline"].astype("category")

model = ols('salary ~ rank * sex * discipline', data=salaries).fit()
anova_table = anova_lm(model)

print(anova_table)

                        df        sum_sq       mean_sq           F  \
rank                   2.0  1.432318e+11  7.161588e+10  138.104573   
sex                    1.0  8.408166e+08  8.408166e+08    1.621437   
discipline             1.0  1.828318e+10  1.828318e+10   35.257420   
rank:sex               2.0  2.351988e+08  1.175994e+08    0.226780   
rank:discipline        2.0  4.686632e+08  2.343316e+08    0.451887   
sex:discipline         1.0  4.619741e+08  4.619741e+08    0.890874   
rank:sex:discipline    2.0  1.323930e+08  6.619650e+07    0.127654   
Residual             385.0  1.996466e+11  5.185627e+08         NaN   

                           PR(>F)  
rank                 6.108879e-46  
sex                  2.036599e-01  
discipline           6.457190e-09  
rank:sex             7.972029e-01  
rank:discipline      6.367634e-01  
sex:discipline       3.458325e-01  
rank:sex:discipline  8.801953e-01  
Residual                      NaN  


# Task 4: Verify the Hypothesis

Objective: Verify if credit amounts (in DM) are significantly different for people applying with different job, personal status, sex, or age.

In [26]:
import pandas as pd

# Load the GermanCredit dataset from GitHub
url = "https://raw.githubusercontent.com/selva86/datasets/master/GermanCredit.csv"
germancredit = pd.read_csv(url)

# Display the first few rows of the dataset
germancredit.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,... < 100 DM,6,critical account/other credits existing,domestic appliances,1169,unknown/no savings account,... >= 7 years,4,male : single,none,...,real estate,67,none,own,2,skilled employee/official,1,yes,yes,1
1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,none,...,real estate,22,none,own,1,skilled employee/official,1,no,yes,0
2,no checking account,12,critical account/other credits existing,retraining,2096,... < 100 DM,4 <= ... < 7 years,2,male : single,none,...,real estate,49,none,own,1,unskilled - resident,2,no,yes,1
3,... < 100 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,guarantor,...,building society savings agreement/life insurance,45,none,for free,1,skilled employee/official,2,no,yes,1
4,... < 100 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,none,...,unknown/no property,53,none,for free,2,skilled employee/official,2,no,yes,0


In [27]:
germancredit.columns

Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk'],
      dtype='object')

# Task 5: Evaluate Interaction Between Group and Time

Description: 

The data provide the anxiety score, measured at three time points, of three groups of individuals practicing physical exercises at different levels (grp1: basal, grp2: moderate and grp3: high)

Objective: Evaluate if there is an interaction between group and time in explaining anxiety scores.

In [13]:
import pandas as pd

# Load the anxiety dataset from GitHub
url = "https://raw.githubusercontent.com/kflisikowski/ds/master/anxiety.csv"
anxiety_data = pd.read_csv(url)

# Display the first few rows of the dataset
print(anxiety_data.head())

   Unnamed: 0  id group    t1    t2    t3
0           1   1  grp1  14.1  14.4  14.1
1           2   2  grp1  14.5  14.6  14.3
2           3   3  grp1  15.7  15.2  14.9
3           4   4  grp1  16.0  15.5  15.3
4           5   5  grp1  16.5  15.8  15.7


In [ ]:
# your solution

# Task 6: Evaluate the Goodness of Fit

Objective: Use the goodness of fit test to determine whether the distribution of credit amounts for male customers matches that of female customers.

In [14]:
import pandas as pd

# Load the German Credit dataset from GitHub
url = "https://raw.githubusercontent.com/selva86/datasets/master/GermanCredit.csv"
germancredit = pd.read_csv(url)

# Display the first few rows of the dataset
print(germancredit.head())

                status  duration                            credit_history  \
0         ... < 100 DM         6   critical account/other credits existing   
1    0 <= ... < 200 DM        48  existing credits paid back duly till now   
2  no checking account        12   critical account/other credits existing   
3         ... < 100 DM        42  existing credits paid back duly till now   
4         ... < 100 DM        24           delay in paying off in the past   

               purpose  amount                     savings  \
0  domestic appliances    1169  unknown/no savings account   
1  domestic appliances    5951                ... < 100 DM   
2           retraining    2096                ... < 100 DM   
3     radio/television    7882                ... < 100 DM   
4            car (new)    4870                ... < 100 DM   

  employment_duration  installment_rate                  personal_status_sex  \
0      ... >= 7 years                 4                        male : single  

In [ ]:
# your solution

# Task 7: Evaluate the Change in Asthma Symptoms Over Time

Objective: determine if there is a significant change in asthma symptoms reported by participants at two different time points.

In [2]:
import pandas as pd

# Load the asthma dataset from GitHub
url = "https://github.com/bougioukas/basic_stats_R/raw/main/data/asthma.xlsx"
asthma_data = pd.read_excel(url)

# Display the first few rows of the dataset
print(asthma_data.head())

  know_begin know_end
0        yes      yes
1         no       no
2        yes       no
3         no       no
4         no       no


In [ ]:
# your solution

# Task 8: Differences of BG readings Over Time 

Objective: determine if there is a significant difference in the blood glucose (BG) readings over multiple time points.

Data: let's use a hypothethical example of blood glucose (BG) readings of persons with diabetes.

The test is done three times, say before, within and after a given clinical treatment and we want to know if there is a significant difference within the groups (times).

In [4]:
# Read dataset from url:
import io
import requests
url="https://raw.githubusercontent.com/trangel/stats-with-python/master/data/BG-db.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),index_col=0)


df.columns=['before','during','after']
df.index.name='Subject'
df.head(10)

,before,during,after
Subject,,,
0,89.162573,94.023517,94.594145
1,90.857629,95.273755,95.040646
2,94.912999,96.612870,95.200472
3,95.254064,96.818673,97.205801
4,97.136291,97.760342,98.428840
5,99.809999,99.169227,98.867769
6,101.094087,99.579283,99.790581
7,101.531428,99.661758,100.669928
8,101.981148,100.812359,101.751155


In [ ]:
# your solution

# Task 9: Evaluate the Change in Mice Weights Before and After Treatment

Objective: determine if there is a significant difference in the weights of mice before and after treatment.

In [34]:
import pandas as pd

# Weight of the mice before treatment
before = [200.1, 190.9, 192.7, 213, 241.4, 196.9, 172.2, 185.5, 205.2, 193.7]

# Weight of the mice after treatment
after = [392.9, 393.2, 345.1, 393, 434, 427.9, 422, 383.9, 392.3, 352.2]

# Create a data frame
my_data = pd.DataFrame({
    'group': ['before'] * len(before) + ['after'] * len(after),
    'weight': before + after
})

# Display the first few rows of the dataset
print(my_data.head(10))

    group  weight
0  before   200.1
1  before   190.9
2  before   192.7
3  before   213.0
4  before   241.4
5  before   196.9
6  before   172.2
7  before   185.5
8  before   205.2
9  before   193.7


In [37]:
before_s = shapiro(before)
after_s = shapiro(after)

print(before_s)
print(after_s)

ShapiroResult(statistic=0.9093793157623515, pvalue=0.2767547422813065)
ShapiroResult(statistic=0.9112065510145709, pvalue=0.28938202337448843)


In [40]:
t_test = ttest_ind(before, after)
alpha = 0.05

print(f"T test results = {t_test}")
if t_test.pvalue < alpha:
    print("Reject NULL hypothesis, there is significant difference")
else:
    print(f"Do not reject NULL hypothesis, there is no significant difference")

T test results = TtestResult(statistic=-17.71377349633779, pvalue=7.744203479129954e-13, df=18.0)
Reject NULL hypothesis, there is significant difference


# Task 10: Calculate Effect Size and Power 

Objective: Use Python to calculate the effect size and power for a test comparing the total bill amounts between smokers and non-smokers. Interpret your results. If the power is not satisfactory - how many observations should we sample to achieve 90% power?

The tips dataset contains information about tips received by waitstaff in a restaurant, including various attributes such as total bill, tip amount, sex of the bill payer, whether the payer is a smoker, day of the week, time of day, and size of the party.

The tips dataset contains the following columns:

total_bill: The total bill amount (including tip) in dollars.

tip: The tip amount in dollars.

sex: The sex of the bill payer (Male or Female).

smoker: Whether the bill payer is a smoker (Yes or No).

day: The day of the week (Thur, Fri, Sat, Sun).

time: The time of day (Lunch or Dinner).

size: The size of the party.

In [42]:
import seaborn as sns
import pandas as pd

# Load the tips dataset
tips = sns.load_dataset('tips')

# Display the first few rows of the dataset
print(tips.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [52]:
shapiro(np.log(total_bill_smoker))

ShapiroResult(statistic=0.9777954587091849, pvalue=0.11395482457978245)

In [53]:
shapiro(np.log(total_bill_non_smoker))

ShapiroResult(statistic=0.99155947421648, pvalue=0.5111865282355945)

In [54]:
total_bill_smoker = np.log(tips[tips["smoker"] != "No"]["total_bill"])
total_bill_non_smoker = np.log(tips[tips["smoker"] == "No"]["total_bill"])

t_test_result_smoker = ttest_ind(total_bill_smoker, total_bill_non_smoker)
alpha = 0.05

print(f"T test results = {t_test_result_smoker}")
if t_test_result_smoker.pvalue < alpha:
    print("Reject NULL hypothesis, there is significant difference")
else:
    print(f"Do not reject NULL hypothesis, there is no significant difference")

T test results = TtestResult(statistic=0.8485524228358794, pvalue=0.39696894352273093, df=242.0)
Do not reject NULL hypothesis, there is no significant difference


In [55]:
import pingouin as pg

pg.compute_effsize(total_bill_smoker, total_bill_non_smoker, eftype='cohen')
print("Cohen's d:", cohen_d)

# Interpretation of effect size
if abs(cohen_d) < 0.2:
    effect_size = "small"
elif abs(cohen_d) < 0.5:
    effect_size = "medium"
else:
    effect_size = "large"

print(f"The effect size (Cohen's d) is {cohen_d:.2f}, which is considered a {effect_size} effect.")

Cohen's d: 0.3164765707641339
The effect size (Cohen's d) is 0.32, which is considered a medium effect.


In [56]:
from statsmodels.stats.power import TTestIndPower

# Calculate the power of the test
analysis = TTestIndPower()
power = analysis.solve_power(effect_size=cohen_d, 
                             nobs1=len(total_bill_smoker), 
                             alpha=alpha, 
                             ratio=len(total_bill_non_smoker)/len(total_bill_smoker), 
                             alternative='two-sided')

print(f"Power of the test: {power:.2f}")
required_n = analysis.solve_power(effect_size=cohen_d, 
                                  power=0.9, 
                                  alpha=alpha, 
                                  ratio=len(total_bill_non_smoker)/len(total_bill_smoker), 
                                  alternative='two-sided')

print(f"Required sample size for 90% power: {int(required_n)} per group")

Power of the test: 0.67
Required sample size for 90% power: 170 per group


# Task 11: 2-way Anova

Objective: Three teachers graded final exams for students, and each exam varied in difficulty (Easy, Medium, Hard). We are interested in investigating whether there are differences in scores based on:
1.	The teacher grading the exam.
2.	The difficulty level of the exam.
3.	The interaction between teacher and difficulty level.


In [59]:
import pandas as pd

# Dane
data = {
    'Teacher': ['Teacher 1'] * 15 + ['Teacher 2'] * 15 + ['Teacher 3'] * 15,
    'Difficulty': ['Easy'] * 5 + ['Medium'] * 5 + ['Hard'] * 5 +
                  ['Easy'] * 5 + ['Medium'] * 5 + ['Hard'] * 5 +
                  ['Easy'] * 5 + ['Medium'] * 5 + ['Hard'] * 5,
    'Scores': [78, 82, 85, 80, 79, 70, 75, 73, 72, 74, 60, 65, 62, 63, 61,
               80, 85, 83, 81, 82, 72, 74, 75, 73, 71, 63, 64, 66, 62, 65,
               81, 83, 82, 84, 85, 73, 71, 74, 72, 70, 64, 63, 62, 65, 66]
}

# Tworzenie DataFrame
df = pd.DataFrame(data)

# Wyświetlenie DataFrame
df.head()

,Teacher,Difficulty,Scores
0,Teacher 1,Easy,78
1,Teacher 1,Easy,82
2,Teacher 1,Easy,85
3,Teacher 1,Easy,80
4,Teacher 1,Easy,79


In [60]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#perform two-way ANOVA
model = ols('Scores ~ C(Teacher) + C(Difficulty) + C(Teacher):C(Difficulty)', data=df).fit()
two_way_anova_table = sm.stats.anova_lm(model, typ=2)

two_way_anova_table

,sum_sq,df,F,PR(>F)
C(Teacher),12.133333,2.0,1.744409,1.891954e-01
C(Difficulty),2594.800000,2.0,373.054313,8.602757e-25
C(Teacher):C(Difficulty),13.866667,4.0,0.996805,4.219076e-01
Residual,125.200000,36.0,NaN,NaN


In [70]:
p_values = two_way_anova_table["PR(>F)"]
for i, p in enumerate(p_values.values):
    if p < 0.05:
        print(f"{p_values.index[i]} have significant difference on Scores, p_value = {p}")
    else:
        print(f"{p_values.index[i]} have NOT significant difference on Scores, p_value = {p}")

C(Teacher) have NOT significant difference on Scores, p_value = 0.1891954187830955
C(Difficulty) have significant difference on Scores, p_value = 8.602756830859398e-25
C(Teacher):C(Difficulty) have NOT significant difference on Scores, p_value = 0.42190761087905604
Residual have NOT significant difference on Scores, p_value = nan


# Task 12: Mantel-Haenszel test

Objective: use the Mantel-Haenszel test to determine if there is a significant difference in the changes in doctoral program completion status between male and female students, controlling for the time variable (initial and final status).

Gdańsk Tech classified students entering the PhD programs in a given year by their status 6 years later, with the data broken down by gender. The initial and final status of students is as follows: Men: 15 completed, 5 not completed, Women: 10 completed, 10 not completed. Final Status: Men: 12 completed, 8 not completed, Women: 8 completed, 12 not completed. Determine if there is a significant difference in the changes of doctoral program completion status between male and female students.


In [4]:
import numpy as np
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

initial = pd.DataFrame([[15, 5],  # Men: Completed, Not completed
                        [10, 10]],  # Women: Completed, Not completed
                       columns=["Completed", "Not completed"],
                       index=["Men", "Women"])

final = pd.DataFrame([[12, 8],  # Men: Completed, Not completed
                      [8, 12]],  # Women: Completed, Not completed
                     columns=["Completed", "Not completed"],
                     index=["Men", "Women"])
change_table = pd.DataFrame(np.zeros((2, 2)), columns=["Completed", "Not completed"], index=["Men", "Women"])

# Fill the change table with differences (completed and not completed)
change_table.loc["Men", "Completed"] = initial.loc["Men", "Completed"] - final.loc["Men", "Completed"]
change_table.loc["Men", "Not completed"] = initial.loc["Men", "Not completed"] - final.loc["Men", "Not completed"]
change_table.loc["Women", "Completed"] = initial.loc["Women", "Completed"] - final.loc["Women", "Completed"]
change_table.loc["Women", "Not completed"] = initial.loc["Women", "Not completed"] - final.loc["Women", "Not completed"]

print("Change Table for Mantel-Haenszel Test:")
print(change_table)

Change Table for Mantel-Haenszel Test:
       Completed  Not completed
Men          3.0           -3.0
Women        2.0           -2.0


In [5]:
from statsmodels.stats.contingency_tables import mcnemar

table = np.array([[initial.loc['Men', 'Completed'], initial.loc['Men', 'Not completed']],
                  [initial.loc['Women', 'Completed'], initial.loc['Women', 'Not completed']]])
result = mcnemar(table)
print(f"p-value of the Mantel-Haenszel test: {result.pvalue}")


p-value of the Mantel-Haenszel test: 0.30175781249999994
